# **FaceFusion WEB UI**

DeepFake AI Tool

**WARNING: COULD RISK YOUR FREE TIER COLAB ACC**

Credits:

- [Nick088](https://linktr.ee/Nick088) (now taking support of the [FaceFusion Unofficial Online Ports](https://github.com/Nick088Official/FaceFusion-Online-Portshttps://github.com/Nick088Official/FaceFusion-Online-Ports).)
- [hina](https://linktr.ee/_hina__) (simplified the code that makes the public gradio link appear, putting share=True without also messing up anything if you cancel execution and re run it.)
- [Original FaceFusion team](https://github.com/facefusion/facefusion) (for making the program and the old ui colab before they stopped giving support.)

## BASIC GUIDE:
1. Select what git_branch to install ('master' is the stable one, 'next' is the beta one) and Run the Install FaceFusion cell and wait until it finishes.
2. Select which type of Tunnel you wanna use, so if the API of one of them is down, you can use the other one:

 A) Gradio: Select it in Tunnel, Run the Run cell and click the Gradio Public Link Url.

 B) Ngrok: Select it in Tunnel, get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new, run the Run cell, wait for the Local URL to appear and click on the Ngrok Tunnel Public URL which is above.

  C) Cloudfare: Select it in Tunnel, run the Run cell, wait for the Local URL to appear and click on the Cloudfare Tunnel Public URL which is above.

  D) LocalTunnel: Select it in Tunnel, run the Run cell, wait for the Local URL to appear, copy the LocalTunnel Password displayed under the public link and paste it in Tunnel Password of the LocalTunnel Tunnel Public URL which is above.

3. For Source select the photo of the person taken as a reference point.
4. For Target, select the photo/video of the person whose face you want to change.
5. **(OPTIONAL)** You can decide to activate the Frame Processors face_enhancer for better results in some cases.
6. Under Face Recognition you can choose Many to change all Target faces or Reference to change only certain reference faces.
7. **(OPTIONAL)** With Reference Face Distance you can choose the radius distance of the reference faces that the program will change.
8. **(OPTIONAL)** Under Reference Analyzer Direction you can choose the radius that identifies the faces to change, from left to right or vice versa.
9. **(OPTIONAL)** Using Face Analyzer Age you can decide whether to change the faces of only people of a certain age.
10. **(OPTIONAL)** Through Face Analyzer Gender you can decide whether to change the faces of only people of a certain sexual gender.
11. **(OPTIONAL)** Under Options you can select Skip Download to automatically download the output.
12. Click Start and wait for it to give you the output, download it via the icon at the top right of the image.


## OFFICIAL FACEFUSION USAGE GUIDES:
- [FaceFusion Docs](https://docs.facefusion.io/knowledgebase)
- [FaceFusion detailed Youtube Video Playlist Tuts](https://www.youtube.com/playlist?list=PLMwq9QXPN0iDReOut5vKekXT4V6wGTET8)


**TIPS:**
- Be careful to make some links every now and then so it doesn't disconnect due to inactivity, you could also check [Google Colab Workarounds](https://docs.google.com/document/d/1Pr-AZndodmWgsbOeuHQU4IrgbatFgYc1ChOq_ZAf_5s/edit?usp=sharing).

- If the process is too slow you can speed it up at the cost of losing some quality of the output, by removing “face_enhancer” and “frame_enhancer” in the frame processors box.

- If the process is too slow, you can split the Target video into videos of 1 minute each, and run the process again for each video, then finally link them all together by placing them on any free editing app like Capcut.


## [CHANGELOG](https://github.com/Nick088Official/FaceFusion-Online-Ports?tab=readme-ov-file#changelog)

In [ ]:
from IPython.display import clear_output
import torch

# Check for CUDA availability
if torch.cuda.is_available():
    device = "cuda"
    # Install NVIDIA CUDA toolkit if GPU is available
    !apt-get install -y nvidia-cuda-toolkit
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

# Direct Git URL and directory name
#giturl = 'https://github.com/sourabh2182/facefusion-master-unlocked.git'
#gitdir = 'facefusion-master-unlocked'

giturl = 'https://github.com/hassan-sd/facefusion-unlocked.git'
gitdir = 'facefusion-unlocked'

# Clone the Git repository
!git clone {giturl}

# Change directory to the cloned repository
%cd /content/{gitdir}

# Run the installation script with the specified options
!python install.py --onnxruntime cuda-11.8 --skip-conda

# Clear output and print completion message
clear_output()
print("Installed! Run Option 1 Or Option 2")


In [ ]:
#@title Run UI
#from IPython.display import clear_output
#import torch
import time
import urllib
import codecs
import re


# Change directory to the cloned repository
%cd /content/facefusion-unlocked

#@markdown The type of tunnel you wanna use for seeing the public link, so if the API of one of them is down, you can use the other one.
Tunnel = "Ngrok" #@param ["Gradio", "Ngrok", "Cloudfare", "LocalTunnel"]

#@markdown Also when using Ngrok, Cloudfare or LocalTunnel as the Tunnel, you need to wait for the Local URL to appear, and only after that click on the  Public URL above it.

#@markdown Use the option under this only if you chose Ngrok as the Tunnel:

#@markdown You can get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new.

ngrok_tunnel_authtoken = "2lbY0JnRCE4PmVITMNlFlDez74H_4K8C2AeKU6tabReHuG818" #@param {type:"string"}

# Update file path for UI based on selected tunnel
file_path_ui = "/content/facefusion-unlocked/facefusion/uis/layouts/default.py"

if Tunnel == "Gradio":
    # Update Gradio configuration to share the UI
    !sed -i 's/show_api = False/show_api=False,share=True/g' $file_path_ui
elif Tunnel == "Ngrok":
    # Update Ngrok configuration
    !sed -i 's/show_api=False,share=True/show_api = False/g' $file_path_ui
    !pip install pyngrok
    from pyngrok import ngrok
    ngrok.set_auth_token(ngrok_tunnel_authtoken)
    http_tunnel = ngrok.connect(7860, bind_tls=True)
    clear_output()
    print("Ngrok Tunnel Public URL:", http_tunnel.public_url)
elif Tunnel == "Cloudfare":
    # Update Cloudflare configuration
    !sed -i 's/show_api=False,share=True/show_api = False/g' $file_path_ui
    !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb
    !rm -rf nohup.out
    !nohup cloudflared tunnel --url localhost:7860 &
    clear_output()
    time.sleep(5)
    cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
    print(f"Cloudflare Tunnel Public URL: {cloudflare_url[0]}")
elif Tunnel == "LocalTunnel":
    # Update LocalTunnel configuration
    !sed -i 's/show_api=False,share=True/show_api = False/g' $file_path_ui
    !npm install -g localtunnel
    with open('url.txt', 'w') as file:
        file.write('')
    get_ipython().system_raw('lt --port 7860 >> url.txt 2>&1 &')
    time.sleep(4)
    endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")
    with open('url.txt', 'r') as file:
        tunnel_url = file.read()
        tunnel_url = tunnel_url.replace("your url is: ", "")
    clear_output()
    print(f"LocalTunnel Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")
    print(f'LocalTunnel Password: {endpoint_ip}')

# Define device and git_branch for running the UI
device = "cuda"  # Set this based on your actual device usage
git_branch = "master"  # Set this based on your actual branch or use a method to determine the branch dynamically

if device == "cuda" and git_branch == "master":
    !python run.py --execution-providers cuda
elif device == "cpu" and git_branch == "master":
    !python run.py --execution-providers cpu
elif device == "cuda" and git_branch == "next":
    !python facefusion-unlocked\.py --execution-providers cuda
elif device == "cpu" and git_branch == "next":
    !python facefusion-unlocked\.py --execution-providers cpu